In [1]:
import openmc
import numpy as np
from openmc.data import AVOGADRO, atomic_mass, atomic_weight

In [2]:
def salt_dens_f(t):
    return 235.0 - (0.02317 * t)

def salt_dens(t):
    return 3.752 - (6.68e-4 * t)

def print_percents(mat):
    at_percents = []
    nucs = []
    at_mass = []
    for nuc, pt, tp in mat.nuclides:
        nucs.append(nuc)
        at_percents.append(pt)
        at_mass.append(atomic_mass(nuc))

    at_percents = np.array(at_percents)
    at_mass = np.array(at_mass)
    print('Atom Percent:')
    for i, nuc in enumerate(nucs):
        print(f'{nuc}: {100 * at_percents[i]}')
        
    mass_percents = at_percents*at_mass / np.dot(at_percents, at_mass)
    print('\nMass Percent:')
    for i, nuc in enumerate(nucs):
        print(f'{nuc}: {mass_percents[i] * 100}')
    
    return nucs, at_percents, mass_percents

In [3]:
salt_dens(900-273.15)

3.3332642

In [4]:
comps = np.array([1/2, 1/2, 1/3, 2/3, 1/5, 4/5, 1/5, 4/5])
vals = np.array([71.7,71.7,16.0,16.0, 12.0,12.0, 0.3,0.3])
nucs = (['Li', 'F', 'Be', 'F', 'Th', 'F', 'U', 'F'])
vals = comps*vals
tots = ([vals[0], vals[1] + vals[3] + vals[5] + vals[7], vals[2], vals[4], vals[6]])
nucs = [nucs[0], nucs[1], nucs[2], nucs[4], nucs[6]]

In [5]:
tots

[35.85, 56.35666666666667, 5.333333333333333, 2.4000000000000004, 0.06]

### Reference Salt

In [6]:
# reference
components = {'Li': {'percent': 0.3585, 
                     'enrichment': 95.995, 
                     'enrichment_target': 'Li7'}, 
              'F19': tots[1]/100, 
              'Be9': tots[2]/100, 
              'Th232': 2.4/100, 
              'U233': 0.06/100}

mat = openmc.Material()
mat.add_components(components, percent_type='ao')
mat

Material
	ID             =	1
	Name           =	
	Temperature    =	None
	Density        =	None [sum]
	S(a,b) Tables  
	Nuclides       
	Li6            =	0.014357924999999968 [ao]
	Li7            =	0.344142075  [ao]
	F19            =	0.5635666666666667 [ao]
	Be9            =	0.05333333333333333 [ao]
	Th232          =	0.024        [ao]
	U233           =	0.0006       [ao]

In [7]:
nucs, at_percents, mass_percents = print_percents(mat)

Atom Percent:
Li6: 1.4357924999999967
Li7: 34.414207499999996
F19: 56.35666666666667
Be9: 5.333333333333333
Th232: 2.4
U233: 0.06

Mass Percent:
Li6: 0.44524487252568634
Li7: 12.447734256968703
F19: 55.198228450282
Be9: 2.4779439297636796
Th232: 28.71000034962854
U233: 0.720848140831377


In [8]:
# Amounts to put in chapter 4 of thesis
for i, nuc in enumerate(nucs):
    print(f'{nuc}: {100 * np.round(mass_percents[i],9)}')
    

Li6: 0.4452449
Li7: 12.4477343
F19: 55.1982285
Be9: 2.4779439
Th232: 28.7100003
U233: 0.7208481


## Model Salt

In [9]:
components = {'Li7': 0.3585, 
              'F19': tots[1]/100, 
              'Be9': tots[2]/100, 
              'Th232': 2.4/100, 
              'U233': 0.06/100}

mat = openmc.Material()
mat.add_components(components, percent_type='ao')
mat

Material
	ID             =	2
	Name           =	
	Temperature    =	None
	Density        =	None [sum]
	S(a,b) Tables  
	Nuclides       
	Li7            =	0.3585       [ao]
	F19            =	0.5635666666666667 [ao]
	Be9            =	0.05333333333333333 [ao]
	Th232          =	0.024        [ao]
	U233           =	0.0006       [ao]

In [10]:
nucs, at_percents, mass_percents = print_percents(mat)

Atom Percent:
Li7: 35.85
F19: 56.35666666666667
Be9: 5.333333333333333
Th232: 2.4
U233: 0.06

Mass Percent:
Li7: 12.957465539473384
F19: 55.15736451584381
Be9: 2.4761094770805
Th232: 28.688745979607145
U233: 0.7203144879951567


In [11]:
#for i, nuc in enumerate(nucs):
#    print(f'{nuc}: {mass_percents[i]}')
    

In [12]:
# Amounts to put in chapter 4 of thesis
for i, nuc in enumerate(nucs):
    print(f'{nuc}: {100 * np.round(mass_percents[i],8)}')
    

Li7: 12.957466
F19: 55.157365000000006
Be9: 2.476109
Th232: 28.688746
U233: 0.720314


## Hastelloy N

In [13]:
components = {'Mo': 0.12, 
              'Cr': 0.07,
              'Fe': 0.03,
              'C': 0.0006,
              'Mn': 0.0035,
              'Si': 0.001,
              'W': 0.001,
              'Al': 0.001,
              'Ti': 0.0125, #avg
              'Cu': 0.001,
              'Co': 0.002,
              'P': 0.00015,
              'S': 0.00015,
              'B': 0.000010,
              'Hf': 0.01,
              'Nb': 0.01}

tot = 0
wts = []
for nuc, wt in components.items():
    wts.append(wt*100)
    tot += wt
nickel = 1 - tot

components.update({'Ni': nickel})

In [14]:
elem_mass = []
elem_wt = []
elems = []
for elem, wt in components.items():
    elem_mass.append(atomic_weight(elem))
    elem_wt.append(wt)
    elems.append(elem)
    
elem_mass = np.array(elem_mass)
elem_wt = np.array(elem_wt)

elem_at = (elem_wt / elem_mass) / (np.sum(elem_wt/elem_mass))


for i, elem in enumerate(elems):
    print(f'{elem}: {elem_wt[i] * 100}')

Mo: 12.0
Cr: 7.000000000000001
Fe: 3.0
C: 0.06
Mn: 0.35000000000000003
Si: 0.1
W: 0.1
Al: 0.1
Ti: 1.25
Cu: 0.1
Co: 0.2
P: 0.015
S: 0.015
B: 0.001
Hf: 1.0
Nb: 1.0
Ni: 73.709


In [15]:
# Amounts to put in chapter 4 of thesis
for i, elem in enumerate(elems):
    print(f'{elem}: {np.round(elem_at[i], 5) * 100}')

Mo: 7.64
Cr: 8.224
Fe: 3.282
C: 0.305
Mn: 0.38899999999999996
Si: 0.218
W: 0.033
Al: 0.22599999999999998
Ti: 1.595
Cu: 0.096
Co: 0.207
P: 0.03
S: 0.029
B: 0.006
Hf: 0.34199999999999997
Nb: 0.658
Ni: 76.72


In [16]:
mat = openmc.Material()
mat.add_components(components, percent_type='wo')
mat

Material
	ID             =	3
	Name           =	
	Temperature    =	None
	Density        =	None [sum]
	S(a,b) Tables  
	Nuclides       
	Mo100          =	0.012175225960709028 [wo]
	Mo92           =	0.016838269656552784 [wo]
	Mo94           =	0.010789582057261462 [wo]
	Mo95           =	0.018840557267706455 [wo]
	Mo96           =	0.01999840362212584 [wo]
	Mo97           =	0.011613114608838517 [wo]
	Mo98           =	0.029744846826805914 [wo]
	Cr50           =	0.002921580575502679 [wo]
	Cr52           =	0.05858955364682328 [wo]
	Cr53           =	0.006771510820966751 [wo]
	Cr54           =	0.001717354956707304 [wo]
	Fe54           =	0.0016936674679200211 [wo]
	Fe56           =	0.027570458631534105 [wo]
	Fe57           =	0.0006481105852150811 [wo]
	Fe58           =	8.776331533079324e-05 [wo]
	C0             =	0.0006       [wo]
	Mn55           =	0.0035       [wo]
	Si28           =	0.0009187351728295046 [wo]
	Si29           =	4.831750292845674e-05 [wo]
	Si30           =	3.294732424203873e-05 [wo